- 데이터는 제공되는 파일인 prob-0101.csv를 사용한다.
- 영화 개수, 배급사 수, 감독 수, 출연진 수, 장르 수를 파악한다.
- 감독, 출연진 등 중복 조건들에 대해 파악한다.
- 본인이 추천하고 싶은 기준을 설정한다.
- 본인의 기준에 의해 영화 몇개를 추천하는 것이 적당한지 실험해 본다.
- 다양한 접근 조회 방법을 실험한다. (감독, 출연진 중복, 장르별 등)
- 개인별로 정한 기준이 서로 어떻게 다른지 확인한다.

In [1]:
import pandas as pd

file_path = './prob-0101.csv'
df = pd.read_csv(file_path)

print(f'전체 데이터 수 : {len(df)}')

column_list = df.columns.tolist()
column_list

전체 데이터 수 : 59


['개봉일', '제목', '배급사', '감독', '출연진', '장르']

In [2]:
import re

without_dup_items = dict()
without_dup_num = dict()

def split_and_eleminate_by_char(item: str, seperator: str) -> set:
    pattern = f"[{re.escape(seperator)}]+"
    item_in_list = re.split(pattern, item)
    item_set = set(item_in_list)
    if (len(item_set) == 1 and '' in item_set):
        return None
    return item_set
    

def split_and_eleminate_set_by_char(item_set: set, seperator: str) -> set:
    result = set()
    for item in item_set:
        new_set = split_and_eleminate_by_char(item, seperator)
        result.update(new_set) if new_set != None else None
    return result



for column in column_list:
    unique_values = df[column].nunique(dropna=True)
    print(f'단순히 중복되지 않은 {column} 수 : {unique_values}')
    without_dup_item = set(df[column])
    if column == "출연진" :
        without_dup_item = split_and_eleminate_set_by_char(without_dup_item, ", ")
    if column == "장르" :
        without_dup_item = split_and_eleminate_set_by_char(without_dup_item, ", ·/")
    print(len(without_dup_item), without_dup_item)
    without_dup_num[column] = len(without_dup_item)
    without_dup_items[column] = without_dup_item
    # print(df[column][~df[column].duplicated(keep=False)])


단순히 중복되지 않은 개봉일 수 : 32
32 {'2023-04-06', '2023-05-24', '2023-05-18', '2023-01-25', '2023-02-23', '2023-06-28', '2023-06-07', '2023-05-17', '2023-06-08', '2023-02-08', '2023-04-26', '2023-01-18', '2023-02-17', '2023-06-14', '2023-04-12', '2023-03-23', '2023-03-01', '2023-01-12', '2023-02-22', '2023-05-10', '2023-03-08', '2023-01-05', '2023-04-27', '2023-04-14', '2023-03-15', '2023-01-04', '2023-04-19', '2023-05-31', '2023-04-05', '2023-06-21', '2023-03-29', '2023-01-20'}
단순히 중복되지 않은 제목 수 : 59
59 {'스프린터', '말이야 바른 말이지', '강남좀비', '안나푸르나', '인드림', '솔라 플라워', '찬란한 나의 복수', '바람개비', '범죄도시 3', '드림', '아수라장: 범털들의 전쟁', '물안에서', '튤립 모양', '육지것들', '정이', '라방', '스위치', '드림팰리스', '리바운드', '살수', '별 볼일 없는 인생', '낭만적 공장', '열여덟, 어른이 되는 나이', '다음 소희', '멍뭉이', '우리 사랑이 향기로 남을 때', '카운트', '흐르다', '나는 여기에 있다', '낭만 캠퍼스', '여섯 개의 밤', '여덟 번째 감각', '유령', '남자는 처음을 원하고 여자는 마지막을 원한다', '소울메이트', '웅남이', '사랑의 고고학', '실버맨', '2퍼센트', '밥만 잘 사주는 이상한 이사님 극장판', '컨버세이션', '장기자랑', '박하경 여행기', '롱디', '불멸의 여자', '익스트림 페스티벌', '대외비', '어쩌면 우린 헤어졌는지 모른다', '

- 감독, 출연진, 장르를 기준으로
- 상위 20%
- 검색 방식 만들기

In [3]:
# 파싱한 새로운 column 추가하기

df['parsed_actor'] = df['출연진'].apply(lambda x: split_and_eleminate_by_char(x, ", "))
df['parsed_genre'] = df['장르'].apply(lambda x: split_and_eleminate_by_char(x, ", ·/"))
df['parsed_director'] = df['감독'].apply(lambda x: split_and_eleminate_by_char(x, ", ·/"))


In [4]:
from typing import TypedDict

class State(TypedDict):
    actor: set[str]
    genre: set[str]
    director: set[str]

def 영화_경험_추천():
    state: State = {}
    state["actor"] = split_and_eleminate_by_char(input("선호하는 배우들을 알려주세요. Ex) 전재준, 박연진, 이사라"), ", ·/")
    state["genre"] = split_and_eleminate_by_char(input("선호하는 장르를 알려주세요. Ex) 로맨스, 공포"), ", ·/")
    state["director"] = split_and_eleminate_by_char(input("선호하는 감독을 알려주세요. Ex) 봉준호"), ", ·/")
    keys = ["actor", "genre", "director"]
    for key in keys:
        if not state.get(key):
            continue
        for value in state.get(key):
            filtered_row = df[df[f"parsed_{key}"].apply(lambda x: value in x)]
            print(f"--- FILTERED {key} ---\n", filtered_row.head(10))

영화_경험_추천()


--- FILTERED actor ---
            개봉일  제목           배급사   감독       출연진          장르 parsed_actor  \
41  2023-05-10  롱디  (주)넥스트엔터테인먼트  임재완  장동윤, 박유나  멜로·로맨스·코미디   {장동윤, 박유나}   

      parsed_genre parsed_director  
41  {로맨스, 멜로, 코미디}           {임재완}  
--- FILTERED genre ---
            개봉일                        제목                 배급사   감독  \
2   2023-01-12                별 볼일 없는 인생            (주)이놀미디어  서동현   
3   2023-01-12                  10일간의 애인              (주)그노스  이영용   
10  2023-02-08           우리 사랑이 향기로 남을 때  (주)콘텐츠존, (주)다자인소프트  임성용   
20  2023-03-29                   여섯 개의 밤            (주)인디스토리  최창환   
21  2023-03-29                  여덟 번째 감각            (주)플레이그램  배인우   
29  2023-04-06                    솔라 플라워              ㈜스토리제이  한경탁   
37  2023-04-19                    낭만적 공장            (주)영화사오윈  조은성   
41  2023-05-10                        롱디        (주)넥스트엔터테인먼트  임재완   
45  2023-05-18  남자는 처음을 원하고 여자는 마지막을 원한다            (주)스토리제이  이현준   

                                  